In [ ]:
%matplotlib inline

from hftcryptoapi import BitmartClient
from hftcryptoapi.bitmart.data import *
from datetime import datetime, timedelta
import pandas as pd
from typing import List, Dict, Any

client = BitmartClient()
symbol = "BTCUSDT"
to_time = datetime.now()
from_time = to_time - timedelta(days=30)
klines = client.get_symbol_kline(symbol=symbol, market=Market.FUTURES, tf=TimeFrame.tf_1h, from_time=from_time, to_time=to_time)


def to_dataframe(kl: List[Kline]) -> pd.DataFrame:
    data = [dict(timestamp=k.date_time, open=k.open, high=k.high, low=k.low, close=k.close) for k in kl]

    df = pd.DataFrame(data=data, columns=["timestamp", "open", "high", "low", "close"])
    df.set_index("timestamp", inplace=True)
    return df

df = to_dataframe(klines)
df

In [ ]:
from ta.momentum import rsi
from ta.volatility import BollingerBands

BB_WINDOW = 20
RSI_WINDOW = 14

def add_indicators(df: pd.DataFrame, rsi_window: int = RSI_WINDOW, bb_window: int = BB_WINDOW):
    df["rsi"] = rsi(df.close, window=rsi_window)
    bb = BollingerBands(df.close, window=bb_window)
    df["bb_upper"] = bb.bollinger_hband()
    df['bb_lower'] = bb.bollinger_lband()
    df.dropna(inplace=True)

    return df

add_indicators(df)
df
df[["close", "bb_upper", "bb_lower"]].plot()

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(2, 1, sharex="row", gridspec_kw={"height_ratios": [5, 1]})

df[["close", "bb_upper", "bb_lower"]].plot(ax=axs[0])
df.rsi.plot(ax=axs[1], label="rsi")
axs[1].legend()
axs[0].get_xaxis().set_visible(False)

def should_buy(r: pd.Series) -> bool:
    return r.rsi < 30 and r.close < r.bb_lower


def should_sell(r: pd.Series) -> bool:
    return r.rsi > 70 and r.close > r.bb_upper

df['buy_signal'] = df.apply(should_buy, axis=1)
df['sell_signal'] = df.apply(should_sell, axis=1)


df_buy = df[df.buy_signal]
df_sell = df[df.sell_signal]

axs[0].scatter(df_buy.index, df_buy.close, marker="^", color="green")
axs[0].scatter(df_sell.index, df_sell.close, marker="v", color="red")

plt.show()




In [ ]:
trades: List[Dict[str, Any]] = []
entry_side = None
entry_price = None

def open(side: Position, price: float):
    global entry_side, entry_price
    entry_side = side
    entry_price = price

def close(price: float, close_time: datetime):
    global entry_side, entry_price
    trades.append(dict(open_price=entry_price, close_price=price, side=entry_side, close_time=close_time))
    entry_side = None
    entry_price = None


for i, row in df[df.buy_signal | df.sell_signal].iterrows():
    if row.buy_signal and entry_side != Position.LONG:
        if entry_side is None:
            open(Position.LONG, row.close)
        else:
            close(row.close, i)

    if row.sell_signal and entry_side != Position.SHORT:
        if entry_side is None:
            open(Position.SHORT, row.close)
        else:
            close(row.close, i)

trades_df = pd.DataFrame(trades)
trades_df.set_index("close_time", inplace=True)

trades_df

In [ ]:
def get_profit(side: Position, open_price: float, close_price: float):
    if side == Position.SHORT:
        return (open_price - close_price) / open_price * 100
    else:
        return (close_price - open_price) / close_price * 100

trades_df["profit"] = trades_df.apply(lambda r: get_profit(r.side, r.open_price, r.close_price), axis=1)

trades_df

In [ ]:
trades_df["profit"].cumsum().plot()

In [ ]:
print("Backtesting summary:")
print(f"Trades: %d" % len(trades_df))
print("Profit Sum: %.3f%%" % trades_df.profit.sum())
print("Max Trade profit.: %.3f%%" % trades_df.profit.max())
print("Min Trade profit.: %.3f%%" % trades_df.profit.min())
print(f"Profit Per Trade(Avg.):  %.3f%%" % trades_df.profit.mean())